In [23]:
%load_ext autoreload
%autoreload 2

import time
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import uniform, randint
from sklearn.preprocessing import MinMaxScaler
import xarray as xr
from keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau
#import pymaise 



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
import psutil
import GPUtil

def get_gpu_utilization():
    # Get the list of available GPUs
    gpus = GPUtil.getGPUs()
    
    if len(gpus) == 0:
        print("No GPU found.")
        return
    
    for gpu in gpus:
        print(f"GPU {gpu.id}:")
        print(f"  - GPU name: {gpu.name}")
        print(f"  - GPU load: {gpu.load * 100:.2f}%")
        print(f"  - GPU memory total: {gpu.memoryTotal} MB")
        print(f"  - GPU memory used: {gpu.memoryUsed} MB")
        print(f"  - GPU memory free: {gpu.memoryFree} MB")
        print(f"  - GPU memory utilization: {gpu.memoryUtil * 100:.2f}%")
        print()

def main():
    # Print CPU utilization
    print("CPU utilization:", psutil.cpu_percent(interval=1))
    
    # Print GPU utilization
    get_gpu_utilization()

if __name__ == "__main__":
    main()


CPU utilization: 0.2
GPU 0:
  - GPU name: NVIDIA GeForce RTX 4090
  - GPU load: 0.00%
  - GPU memory total: 24564.0 MB
  - GPU memory used: 22748.0 MB
  - GPU memory free: 1468.0 MB
  - GPU memory utilization: 92.61%

GPU 1:
  - GPU name: NVIDIA GeForce RTX 4090
  - GPU load: 0.00%
  - GPU memory total: 24564.0 MB
  - GPU memory used: 562.0 MB
  - GPU memory free: 23654.0 MB
  - GPU memory utilization: 2.29%



In [25]:
from pyMAISE.datasets import load_anomaly
from pyMAISE.preprocessing import train_test_split, scale_data, one_hot_encode, SplitSequence
import pyMAISE as mai

In [26]:
%%time

global_settings = mai.init(
    problem_type=mai.ProblemType.CLASSIFICATION,   # Define a regression problem
    #cuda_visible_devices="-1"                  # Use CPU only/ Delete line when run on GPU
    verbosity = 3
)

Num GPUs Available:  2
CPU times: user 94 µs, sys: 0 ns, total: 94 µs
Wall time: 99.9 µs


In [27]:
# call handler
input_path = "/home/jacc/pyMAISE/pyMAISE/datasets/DTL.npy"
output_path = "/home/jacc/pyMAISE/pyMAISE/datasets/DTL_labels.npy"
#print(mai.__file__)
#False, False, False: WORKS       X.shape = (1077, 4500, 14), Y.shape = (1077, 1)  one hot
#False, True, False: WORKS       X.shape = (1077, 4500, 14), Y.shape = (1077, 1)
#False, False, True:  ?      X.shape = (1077, 4500, 14), Y.shape = (1077, 4500, 1)
#False, True, True:   ?      X.shape = (1077, 4500, 14), Y.shape = (1077, 4500, 1)
#True, False, True: WORKS        X.shape = (4846500, 14), Y.shape = (4846500, 1)
#True, True, True: WORKS         X.shape = (4846500, 14), Y.shape = (4846500, 1)
cond1 = True
cond2 = False
cond3 = True
inputs, outputs = load_anomaly([input_path, output_path], cond1, cond2, cond3)


In [28]:
inputs

<xarray.DataArray (timesteps: 4846500, features: 14)>
array([[-6.07100e+00,  5.55120e+00, -1.42220e+00, ..., -4.60370e-02,
         2.17170e+03,  1.27100e+00],
       [-4.85030e+00,  4.33040e+00, -2.64290e+00, ...,  1.50000e-02,
         2.17250e+03, -1.33776e+01],
       [-4.85030e+00,  1.88890e+00, -1.42220e+00, ...,  4.55190e-02,
         2.17320e+03, -8.49500e+00],
       ...,
       [-2.17470e+00, -7.57620e+00, -3.39550e+00, ..., -1.07070e-01,
         2.17220e+03,  1.19730e+01],
       [-4.61620e+00, -3.91400e+00, -5.83700e+00, ..., -1.68110e-01,
         2.17180e+03,  9.53200e+00],
       [-3.39550e+00, -6.35550e+00, -5.83700e+00, ..., -4.60370e-02,
         2.17150e+03,  9.53200e+00]], dtype=float32)
Coordinates:
  * timesteps  (timesteps) int64 0 1 2 3 4 ... 4846496 4846497 4846498 4846499
  * features   (features) <U18 'A+IGBT-I: current' ... 'DV/DT'

In [29]:
outputs

<xarray.DataArray (timesteps: 4846500, features: 1)>
array([['Run'],
       ['Run'],
       ['Run'],
       ...,
       ['Fault'],
       ['Fault'],
       ['Fault']], dtype=object)
Coordinates:
  * timesteps  (timesteps) int64 0 1 2 3 4 ... 4846496 4846497 4846498 4846499
  * features   (features) <U5 'Class'

In [30]:
outputs = one_hot_encode(outputs)
outputs

<xarray.DataArray (timesteps: 4846500, features: 2)>
array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])
Coordinates:
  * timesteps  (timesteps) int64 0 1 2 3 4 ... 4846496 4846497 4846498 4846499
  * features   (features) object 'Class_Fault' 'Class_Run'

In [31]:
data = xr.concat([inputs, outputs], dim="features")
data

<xarray.DataArray (timesteps: 4846500, features: 16)>
array([[ -6.0710001 ,   5.55119991,  -1.42219996, ...,   1.27100003,
          0.        ,   1.        ],
       [ -4.85029984,   4.33039999,  -2.64289999, ..., -13.37759972,
          0.        ,   1.        ],
       [ -4.85029984,   1.88890004,  -1.42219996, ...,  -8.49499989,
          0.        ,   1.        ],
       ...,
       [ -2.17470002,  -7.57620001,  -3.39549994, ...,  11.97299957,
          1.        ,   0.        ],
       [ -4.61619997,  -3.91400003,  -5.83699989, ...,   9.53199959,
          1.        ,   0.        ],
       [ -3.39549994,  -6.35550022,  -5.83699989, ...,   9.53199959,
          1.        ,   0.        ]])
Coordinates:
  * timesteps  (timesteps) int64 0 1 2 3 4 ... 4846496 4846497 4846498 4846499
  * features   (features) object 'A+IGBT-I: current' ... 'Class_Run'

In [32]:
split_sequence = SplitSequence(
    10, 
    1, 
    0, 
    sequence_inputs=data.coords["features"].values[:-2], 
    sequence_outputs=data.coords["features"].values[-2:],
)
inputs, outputs = split_sequence.split(data)

inputs

<xarray.DataArray (samples: 4846491, timesteps: 10, features: 14)>
array([[[-6.07100010e+00,  5.55119991e+00, -1.42219996e+00, ...,
         -4.60369997e-02,  2.17169995e+03,  1.27100003e+00],
        [-4.85029984e+00,  4.33039999e+00, -2.64289999e+00, ...,
          1.49999997e-02,  2.17250000e+03, -1.33775997e+01],
        [-4.85029984e+00,  1.88890004e+00, -1.42219996e+00, ...,
          4.55190018e-02,  2.17319995e+03, -8.49499989e+00],
        ...,
        [-3.62960005e+00,  1.88890004e+00, -5.08440018e+00, ...,
         -4.60369997e-02,  2.17239990e+03, -1.17100000e+00],
        [-7.29180002e+00,  1.88890004e+00, -3.86360002e+00, ...,
          1.49999997e-02,  2.17230005e+03, -1.33781004e+01],
        [-7.29180002e+00,  3.10969996e+00, -2.64289999e+00, ...,
         -4.60369997e-02,  2.17280005e+03, -1.17100000e+00]],

       [[-4.85029984e+00,  4.33039999e+00, -2.64289999e+00, ...,
          1.49999997e-02,  2.17250000e+03, -1.33775997e+01],
        [-4.85029984e+00,  1.88890004e+00, -1.42219996e+00, ...,
          4.55190018e-02,  2.17319995e+03, -8.49499989e+00],
        [-7.29180002e+00,  1.88890004e+00, -1.42219996e+00, ...,
          4.55190018e-02,  2.17300000e+03, -2.31439991e+01],
...
        [-4.61619997e+00, -8.79699993e+00, -4.61619997e+00, ...,
          1.49999997e-02,  2.17180005e+03,  4.64900017e+00],
        [-2.17470002e+00, -7.57620001e+00, -3.39549994e+00, ...,
         -1.07069999e-01,  2.17219995e+03,  1.19729996e+01],
        [-4.61619997e+00, -3.91400003e+00, -5.83699989e+00, ...,
         -1.68109998e-01,  2.17180005e+03,  9.53199959e+00]],

       [[-7.05770016e+00, -7.57620001e+00, -3.39549994e+00, ...,
          1.06559999e-01,  2.17180005e+03,  7.09000015e+00],
        [-4.61619997e+00, -6.35550022e+00, -7.05770016e+00, ...,
         -4.60369997e-02,  2.17160010e+03,  7.09100008e+00],
        [-3.39549994e+00, -8.79699993e+00, -2.17470002e+00, ...,
         -4.60369997e-02,  2.17180005e+03,  1.19740000e+01],
        ...,
        [-2.17470002e+00, -7.57620001e+00, -3.39549994e+00, ...,
         -1.07069999e-01,  2.17219995e+03,  1.19729996e+01],
        [-4.61619997e+00, -3.91400003e+00, -5.83699989e+00, ...,
         -1.68109998e-01,  2.17180005e+03,  9.53199959e+00],
        [-3.39549994e+00, -6.35550022e+00, -5.83699989e+00, ...,
         -4.60369997e-02,  2.17150000e+03,  9.53199959e+00]]])
Coordinates:
  * samples    (samples) int64 0 1 2 3 4 ... 4846487 4846488 4846489 4846490
  * timesteps  (timesteps) int64 0 1 2 3 4 5 6 7 8 9
  * features   (features) object 'A+IGBT-I: current' ... 'DV/DT'

In [33]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler






In [34]:
x_train, x_test, y_train, y_test = train_test_split(inputs, outputs, test_size=0.3)



In [35]:
x_train_scaled, x_test_scaled, x_scaler = scale_data(x_train, x_test, scaler=MinMaxScaler())



In [36]:
y_train_scaled, y_test_scaled, y_scaler = scale_data(y_train, y_test, scaler=MinMaxScaler())

In [37]:
%%time


x_train_sampled = x_train_scaled[:8000]
y_train_sampled = y_train_scaled[:8000]

x_test_sampled = x_test_scaled[:2000]
y_test_sampled = y_test_scaled[:2000]


CPU times: user 0 ns, sys: 5.25 ms, total: 5.25 ms
Wall time: 5.21 ms


In [38]:
x_train_sampled.shape

(8000, 10, 14)

In [39]:
y_train_sampled.shape[-1]

2

In [40]:
lstm_structure = {
    "LSTM_input": {
        "units": 50,
        "return_sequences": True,
        "input_shape": (x_train_scaled.shape[1], x_train_scaled.shape[2]),
    },
    "LSTM": {
        "num_layers": mai.Int(min_value=0, max_value=3),
        "units": 50,
        "return_sequences": True,
        "activation": "tanh",
        "recurrent_activation": "sigmoid",
    },
#check activation to lstm to classification
    "LSTM_output": {
        "units": 50,
        "activation": "tanh",
        "recurrent_activation": "sigmoid",

        
    },
    "Dense":{
         "num_layers": mai.Int(min_value=0, max_value=3),
         "units": mai.Int(min_value=25, max_value=250),
         "activation": "sigmoid", #do research on relu activation CHANGED
         "kernel_initializer": "normal",
         "sublayer": mai.Choice(["Dropout_hidden", "None"]),
         "Dropout_hidden": {
         "rate": mai.Float(min_value=0.4, max_value=0.6),}
    },
    "Dense_output": {
        "units": y_train_scaled.shape[1],
        "activation": "softmax",
    },
}
#change nodes in each layer, not in the output
fitting = {
    "batch_size":128,# mai.Choice([16, 32, 64]),
    "epochs": 5,
    "validation_split":0.15,
    "callbacks": [
        ReduceLROnPlateau(
            monitor='accuracy',
            factor=0.8,
            patience=2,
            min_lr=0,
            verbose=1,
        ),
        EarlyStopping(
            monitor="accuracy",
            patience=3,
        )
    ]
}
adam = {
    "learning_rate": mai.Float(min_value=0.00001, max_value=0.001),
   # "clipnorm": mai.Float(min_value=0.8, max_value=1.2),
  #  "clipvalue": mai.Float(min_value=0.3, max_value=0.7),
}
compiling = {
    "loss": "binary_crossentropy",#figure out accuracy on keras keras lstm on classification
    "metrics": ["accuracy"],
}

model_settings = {
    "models": ["LSTM"],
    "LSTM": {
        "structural_params": lstm_structure,
        "optimizer": "Adam",
        "Adam": adam,
        "compile_params": compiling,
        "fitting_params": fitting,
    },
}
tuner = mai.Tuner(x_train_sampled, y_train_sampled, model_settings=model_settings)

In [41]:
x_train_sampled.shape

(8000, 10, 14)

In [42]:
lstm_structure

{'LSTM_input': {'units': 50,
  'return_sequences': True,
  'input_shape': (10, 14)},
 'LSTM': {'num_layers': <pyMAISE.utils.hyperparameters.Int at 0x722ab35511d0>,
  'units': 50,
  'return_sequences': True,
  'activation': 'tanh',
  'recurrent_activation': 'sigmoid'},
 'LSTM_output': {'units': 50,
  'activation': 'tanh',
  'recurrent_activation': 'sigmoid'},
 'Dense': {'num_layers': <pyMAISE.utils.hyperparameters.Int at 0x722af0f7f450>,
  'units': <pyMAISE.utils.hyperparameters.Int at 0x722af0f7d110>,
  'activation': 'sigmoid',
  'kernel_initializer': 'normal',
  'sublayer': <pyMAISE.utils.hyperparameters.Choice at 0x722b88d41c50>,
  'Dropout_hidden': {'rate': <pyMAISE.utils.hyperparameters.Float at 0x722ab35520d0>}},
 'Dense_output': {'units': 2, 'activation': 'softmax'}}

In [43]:
y_train_sampled.shape

(8000, 2)

In [44]:

start = time.time()

bayesian_search_configs = tuner.nn_bayesian_search(
    objective="accuracy_score",
    max_trials=3,
    cv=5,
)

print("Hyperparameter tuning took " + str((time.time() - start) / 60) + " minutes to process.")

Trial 3 Complete [00h 00m 20s]
accuracy_score: 0.85775

Best accuracy_score So Far: 0.87775
Total elapsed time: 00h 01m 16s

Top Configurations

-- LSTM | Training Time: 00:01:15
LSTM_num_layers: 0
Dense_num_layers: 0
Adam_learning_rate: 0.0006807002275779613
Hyperparameter tuning took 1.262358804543813 minutes to process.


In [45]:
x_train_scaled

<xarray.DataArray (samples: 3392543, timesteps: 10, features: 14)>
array([[[0.59440541, 0.29775133, 0.15822146, ..., 0.51764699,
         0.9169071 , 0.42174107],
        [0.59593355, 0.25922157, 0.17670308, ..., 0.52498312,
         0.91622898, 0.41084081],
        [0.59746169, 0.22515257, 0.19026974, ..., 0.52039935,
         0.91781127, 0.40841302],
        ...,
        [0.59929526, 0.09331141, 0.25971884, ..., 0.49793199,
         0.91713314, 0.30684239],
        [0.59134939, 0.07293783, 0.26712495, ..., 0.49884904,
         0.9195744 , 0.28504186],
        [0.60051777, 0.06529447, 0.29136313, ..., 0.49701494,
         0.91586734, 0.26809692]],

       [[0.59287752, 0.35252515, 0.79888775, ..., 0.53415365,
         0.9396469 , 0.2669078 ],
        [0.59104371, 0.359532  , 0.79518907, ..., 0.52865193,
         0.93860709, 0.26928603],
        [0.59104371, 0.36844836, 0.79025761, ..., 0.51856494,
         0.93942086, 0.2705247 ],
...
        [0.60387953, 0.61174283, 0.80012056, ..., 0.49701494,
         0.91428505, 0.54754497],
        [0.60418513, 0.610469  , 0.80012056, ..., 0.49793199,
         0.91374252, 0.5439132 ],
        [0.60387953, 0.61015056, 0.80012056, ..., 0.49793199,
         0.91347128, 0.5439132 ]],

       [[0.59782595, 0.06312934, 0.35603197, ..., 0.48967896,
         0.92472816, 0.45726602],
        [0.59660349, 0.0583555 , 0.36589554, ..., 0.48784485,
         0.92545148, 0.4657385 ],
        [0.59324178, 0.05932068, 0.37700471, ..., 0.48601135,
         0.92522544, 0.47426052],
        ...,
        [0.58804626, 0.10262428, 0.43659023, ..., 0.49242997,
         0.92508985, 0.50929   ],
        [0.58712948, 0.10930242, 0.44850735, ..., 0.51031161,
         0.92486381, 0.50929   ],
        [0.58896304, 0.12046745, 0.46247796, ..., 0.5213158 ,
         0.92531589, 0.50324531]]])
Coordinates:
  * samples    (samples) int64 3431767 1576341 3309482 ... 422455 3670082
  * timesteps  (timesteps) int64 0 1 2 3 4 5 6 7 8 9
  * features   (features) object 'A+IGBT-I: current' ... 'DV/DT'

In [46]:
y_train_scaled.shape

(3392543, 2)

In [47]:
postprocessor = mai.PostProcessor(
    data=(x_train_sampled, x_test_sampled, y_train_sampled, y_test_sampled), 
    model_configs=[bayesian_search_configs], 
    new_model_settings={
        "FNN": {"fitting_params": {"epochs": 10}},
    },
  #  y_scaler=y_scaler,
)

Epoch 1/5
Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 5/5
63/63 [==============================] - 0s 1ms/step
Epoch 1/5
Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 5/5
63/63 [==============================] - 1s 3ms/step
Epoch 1/5
Epoch 2/5
Epoch 3/5

Epoch 3: ReduceLROnPlateau reducing learning rate to 5.263937055133283e-05.
Epoch 4/5
63/63 [==============================] - 1s 1ms/step


In [48]:
postprocessor.metrics()

,Model Types,Parameter Configurations,Train Accuracy,Train Recall,Train Precision,Train F1,Test Accuracy,Test Recall,Test Precision,Test F1
0,LSTM,"{'LSTM_num_layers': 0, 'Dense_num_layers': 0, ...",0.883625,0.883625,0.883625,0.883625,0.891,0.891,0.891,0.891
1,LSTM,"{'LSTM_num_layers': 2, 'Dense_num_layers': 3, ...",0.857750,0.857750,0.857750,0.857750,0.858,0.858,0.858,0.858
2,LSTM,"{'LSTM_num_layers': 0, 'Dense_num_layers': 2, ...",0.857750,0.857750,0.857750,0.857750,0.858,0.858,0.858,0.858


In [57]:
for model in [ "LSTM"]:
    for key, value in postprocessor.get_params(model_type=model).to_dict().items():
        print(f"{key}: {value[0]}")
    print()

Model Types: LSTM
LSTM_num_layers: 0
Dense_num_layers: 0
Adam_learning_rate: 0.0006807002275779613



In [52]:
postprocessor.confusion_matrix(  ax=None,
        idx=None,
        model_type=None,
        sort_by=accuracy_score,
        direction=None)


NameError: name 'accuracy_score' is not defined